In [1]:
from PIL import Image
import scipy as sp
import numpy as np
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

In [3]:
""" Image Preprocessing
Our goal here is to create a sparse matrix of shape (# pixels, 3) where
each row represents each pixel (ordered row wise) and each column repr-
-esents each of R,G,B values. It should also be a sparse matrix.
"""
image = Image.open('./test_image.png').convert('RGB')
image_array = np.array(image).reshape(-1, 3)
sp_image_array = sp.sparse.csr_matrix(image_array)

np: 125952
coo: 1133505
csr: 797665


In [6]:
""" Spectral Clustering
"""
spec = SpectralClustering(n_clusters = 5)
pred = spec.fit_predict(sp_image_array)
pred.shape

MemoryError: Unable to allocate 13.1 GiB for an array with shape (1762656256,) and data type float64